# CS349: Machine Learning
# Final project: Cryptocurrenty price prediction with LSTM
## Chanwook Park


In [1]:
import json
import requests
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pytz
import pyupbit

## pyupbit: API of Upbit, the largest crypticurrency exchange platform in South Korea.
## LSTM algorithm: Get 10 past ohlcv data and predict 3 future close prices.

In [2]:
# Initial Settings

my_interval = 'day'
hist = pyupbit.get_ohlcv('KRW-ETH', interval=my_interval, count=1000, period=1)
target_col = 'close'
target_col_num = 3

np.random.seed(42)
window_len_X = 10
window_len_y = 3
test_size = 0.1
lstm_neurons = 100
epochs = 10
batch_size = 32
loss = 'mse'
dropout = 0.2
optimizer = 'adam'

## MinMaxScalar normalization

In [3]:
# Normalization

X_all_o = hist.values
y_all_o = hist[target_col].values.reshape(-1,1)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_all_n = scaler_X.fit_transform(X_all_o)
y_all_n = scaler_y.fit_transform(y_all_o)

## Functions

In [4]:
def train_test_split(df, test_size=0.2):
    split_row = len(df) - int(test_size * len(df))
    train_data = df[:split_row]
    test_data = df[split_row:]
    return train_data, test_data


def line_plot(line1, line2, label1=None, label2=None, title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(13, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel('price [KRW]', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16);
    ax.grid(axis='y')
    plt.xticks(rotation=90)
    
def extract_window_data(df, window_len_X = 5, window_len_y = 3):
    window_data = []
    for idx in range(len(df) - window_len_X - window_len_y + 1):
        tmp = df[idx: (idx + window_len_X), :]
        window_data.append(tmp)
    return np.array(window_data)

def extract_window_data_y(df, window_len_X = 5, window_len_y = 3):
    window_data = []
    for idx in range(len(df) - window_len_X - window_len_y + 1):
        tmp = df[idx + window_len_X: (idx + window_len_X + window_len_y), target_col_num]
        window_data.append(tmp)
    return np.array(window_data)

def prepare_data(df, target_col_num, window_len_X = 5, window_len_y = 3, test_size=0.2):
    train_data, test_data = train_test_split(df, test_size=test_size)
    
    X_train = extract_window_data(train_data, window_len_X, window_len_y)
    X_test = extract_window_data(test_data, window_len_X, window_len_y)
    
    y_train = extract_window_data_y(train_data, window_len_X, window_len_y)
    y_test = extract_window_data_y(test_data, window_len_X, window_len_y)
    
    return train_data, test_data, X_train, X_test, y_train, y_test    

def build_lstm_model(input_data, output_size, neurons=100, activ_func='linear',
                     dropout=0.2, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(input_data.shape[1], input_data.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    model.summary()

    model.compile(loss=loss, optimizer=optimizer)
    return model

# LSTM procedure

In [5]:
# Prepare train and test data
train_n, test_n, X_train_n, X_test_n, y_train_n, y_test_n = prepare_data(
    X_all_n, target_col_num, window_len_X, window_len_y, test_size)

In [6]:
# Model generation
model = build_lstm_model(
    X_train_n, output_size=window_len_y, neurons=lstm_neurons, dropout=dropout, loss=loss, optimizer=optimizer)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               42800     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
_________________________________________________________________
activation (Activation)      (None, 3)                 0         
Total params: 43,103
Trainable params: 43,103
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Train
history = model.fit(X_train_n, y_train_n, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

Epoch 1/10
28/28 [==============================] - 0s 5ms/step - loss: 0.0015
Epoch 2/10
28/28 [==============================] - 0s 4ms/step - loss: 2.4931e-04
Epoch 3/10
28/28 [==============================] - 0s 4ms/step - loss: 2.3879e-04
Epoch 4/10
28/28 [==============================] - 0s 4ms/step - loss: 2.3024e-04
Epoch 5/10
28/28 [==============================] - 0s 4ms/step - loss: 2.1131e-04
Epoch 6/10
28/28 [==============================] - 0s 4ms/step - loss: 2.1241e-04
Epoch 7/10
28/28 [==============================] - 0s 4ms/step - loss: 1.6749e-04
Epoch 8/10
28/28 [==============================] - 0s 4ms/step - loss: 1.7754e-04
Epoch 9/10
28/28 [==============================] - 0s 4ms/step - loss: 1.7024e-04
Epoch 10/10
28/28 [==============================] - 0s 4ms/step - loss: 1.5199e-04


In [ ]:
#%% Visualize Test results
train_data, test_data = train_test_split(hist, test_size = test_size)
targets = test_data[target_col][window_len_X:]

for i in range( len(X_test_n) ):
    # i = 85 
    X_test_n_i = X_test_n[i:i+1,:,:].copy()
    y_test_pred_n = model.predict(X_test_n_i).squeeze()
    y_test_pred_o = scaler_y.inverse_transform(y_test_pred_n.reshape(-1,1))
    # y_test_o = scaler_y.inverse_transform(y_test_n.reshape(-1,1))
    
    targets_i = targets[ i : i + window_len_y]
    
    preds = pd.Series(index=targets_i.index, data = y_test_pred_o[:,0])
    line_plot(targets, preds, 'actual', 'prediction', lw=3)

In [ ]:
#%% Visualize Train results
train_data, test_data = train_test_split(hist, test_size = test_size)
targets = train_data[target_col][window_len_X:]

for i in range( len(X_train_n) ):
    # i = 85 
    X_train_n_i = X_train_n[i:i+1,:,:].copy()
    y_train_pred_n = model.predict(X_train_n_i).squeeze()
    y_train_pred_o = scaler_y.inverse_transform(y_train_pred_n.reshape(-1,1))
    # y_test_o = scaler_y.inverse_transform(y_test_n.reshape(-1,1))
    
    targets_i = targets[ i : i + window_len_y]
    
    preds = pd.Series(index=targets_i.index, data = y_train_pred_o[:,0])
    line_plot(targets, preds, 'actual', 'prediction', lw=3)